In [2]:
from sklearn.datasets import fetch_openml
import numpy as np
mnist = fetch_openml('mnist_784', version=1)

/home/makowskitomasz/anaconda3/envs/ml/lib/python3.10/site-packages/sklearn/datasets/_openml.py:932: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [23]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
import time
import pickle
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

In [4]:
print(mnist.keys())
print(mnist["target"][0])

dict_keys(['data', 'target', 'frame', 'categories', 'feature_names', 'target_names', 'DESCR', 'details', 'url'])
5


In [5]:
print((np.array(mnist.data.loc[0]).reshape(28, 28) > 0).astype(int))

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0]
 [0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 0 1 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 

In [6]:
X, y = mnist["data"], mnist["target"].astype(np.uint8)
y = y.sort_values(ascending=True)
X = X.reindex(y.index)

In [7]:
X_train, X_test = X[:56000], X[56000:]
y_train, y_test = y[:56000], y[56000:]
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(56000, 784) (56000,)
(14000, 784) (14000,)


In [8]:
print(np.unique(y_train))
print(np.unique(y_test))

[0 1 2 3 4 5 6 7]
[7 8 9]


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(56000, 784) (56000,)
(14000, 784) (14000,)


In [10]:
y_train_0 = (y_train == 0)
y_test_0 = (y_test == 0)
print(y_train_0.head())
print(np.unique(y_train_0))
print(len(y_train_0))

32576    False
10306    False
37102    False
57835    False
15160     True
Name: class, dtype: bool
[False  True]
56000


In [11]:
start = time.time()
sgd_classifier = SGDClassifier(random_state=42)
sgd_classifier.fit(X_train, y_train_0)
print(time.time() - start)

6.0886311531066895


In [12]:
start = time.time()
print(sgd_classifier.predict([mnist.data.loc[0], mnist.data.loc[1]]))
print(time.time()-start)

[False  True]
0.007269144058227539


/home/makowskitomasz/anaconda3/envs/ml/lib/python3.10/site-packages/sklearn/base.py:409: UserWarning: X does not have valid feature names, but SGDClassifier was fitted with feature names
  warnings.warn(


In [13]:
y_train_pred = sgd_classifier.predict(X_train)
y_test_pred = sgd_classifier.predict(X_test)

acc_train = sum(y_train_pred == y_train_0)/len(y_train_0)
acc_test = sum(y_test_pred == y_test_0)/len(y_test_0)

print(acc_train, acc_test)
acc_list = [acc_train, acc_test]
print(acc_list)

with open('sgd_acc.pkl', 'wb') as f:
    pickle.dump(acc_list, f)


0.9832142857142857 0.9818571428571429
[0.9832142857142857, 0.9818571428571429]


In [14]:
start = time.time()
score = cross_val_score(sgd_classifier, X_train, y_train_0, cv=3, scoring="accuracy", n_jobs=-1)
print(time.time() - start)
print(score)

7.263622760772705
[0.98842878 0.98805378 0.98799957]


In [17]:
sgd_m_clf = SGDClassifier(random_state=42,n_jobs=-1)
sgd_m_clf.fit(X_train, y_train)
print(sgd_m_clf.predict([mnist.data.loc[0],
mnist.data.loc[1]]))

[5 0]


/home/makowskitomasz/anaconda3/envs/ml/lib/python3.10/site-packages/sklearn/base.py:409: UserWarning: X does not have valid feature names, but SGDClassifier was fitted with feature names
  warnings.warn(


In [24]:
print(cross_val_score(sgd_m_clf, X_train, y_train, cv=3, scoring="accuracy", n_jobs=-1))
y_train_pred = cross_val_predict(sgd_m_clf, X_train, y_train, cv=3, n_jobs=-1)
confusion_matrix = confusion_matrix(y_train, y_train_pred)
print(confusion_matrix)

[0.88048428 0.87132373 0.8772099 ]
[[5295    2   22   42   16   26   37    6   71   21]
 [   3 6098   39   19    4   18    6    9   64   25]
 [  53   74 4798  131   65   18   86   65  273   38]
 [  41   30  157 4888   12  179   24   50  260  106]
 [  22   30   30   14 4691   22   69   23  156  401]
 [  93   29   41  251   67 3806  113   20  477  106]
 [  53   28   45   10   59   63 5114    5   92    8]
 [  27   32   48   40   67   10    6 5154  108  358]
 [  49  122   67  146   32  288   35   15 4518  183]
 [  31   29   18   66  287   42    5  217  178 4713]]


In [26]:
with open('sgd_cva.pkl', 'wb') as f:
    pickle.dump(score, f)

In [27]:
with open('sgd_cmx.pkl', 'wb') as f:
    pickle.dump(confusion_matrix, f)